# Basic Convolutio Layer on Fashion MNIST

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as ipw
import tensorflow as tf
from ipywidgets import interactive
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.gridspec as gridspec

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/')
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

## Neural net inputs

#### Image

In [ ]:
batch_size = 10
images = data.validation.images[:batch_size]
# images, _ = data.train.next_batch(batch_size)

In [ ]:
image_id = 1
assert image_id < batch_size
image = images[image_id].reshape(28,28)

plt.figure(figsize=(10, 10))
im = plt.imshow(image, cmap='gray')
plt.colorbar(im, orientation='horizontal')
plt.gca().axes.set_axis_off()

#### Convolution filter

In [ ]:
conv_filter = np.array(
    [[-1, -1, -1, 1], 
     [-1, -1, 1, 1], 
     [-1, 1, 1, 1],
     [1, 1, 1, 1]], dtype=np.float32)

plt.figure(figsize=(10, 10))
im = plt.imshow(conv_filter, cmap='gray')
plt.colorbar(im, orientation='horizontal')
ax = plt.gca()
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())


## Parameters Setup

#### Stride and Padding

In [ ]:
stride_row = 1
stride_col = 1
padding = "SAME"

#### Image and Filter shape

In [ ]:
conv_filter.shape

In [ ]:
image.shape

## Run Convolution Net

In [ ]:
image_reshaped = image.reshape((1, 28, 28, 1))
conv_filter_reshaped = conv_filter.reshape(conv_filter.shape[0], conv_filter.shape[1], 1, 1)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    tf_image = tf.constant(image_reshaped)
    tf_conv_filer = tf.constant(conv_filter_reshaped)
    tf_conv_layer = tf.nn.conv2d(tf_image, tf_conv_filer, strides=[1, stride_row, stride_col, 1], padding=padding)

In [ ]:
with tf.Session(graph=graph) as sess:
    conv_layer = sess.run(tf_conv_layer)[0]

## Observe Convolution Layer Output Shape

In [ ]:
conv_layer.shape[:2]

In [ ]:
if padding == 'VALID':
    conv_layer_shape = (np.floor((28 - conv_filter.shape[0] + stride_row) / stride_row).astype(int), 
                        np.floor((28. - conv_filter.shape[1] + stride_col) / stride_col).astype(int))
elif padding == 'SAME':
    conv_layer_shape = (np.ceil(28 / stride_row).astype(int), 
                        np.ceil(28 /stride_col).astype(int))
conv_layer_shape

## Visualizations

In [ ]:
conv_layer = conv_layer.reshape(conv_layer_shape)

#### Input vs. Convolution Layer

In [ ]:
fig = plt.figure(figsize=(16, 16))
gs = gridspec.GridSpec(2, 2)

ax = plt.subplot(gs[0, 0])
plt.title("Convolved")
im = plt.imshow(conv_layer, cmap='gray')
plt.gca().axes.set_axis_off()

ax = plt.subplot(gs[0, 1])
plt.title('Input')
im = plt.imshow(image, cmap='gray')
plt.gca().axes.set_axis_off()


ax = plt.subplot(gs[1, :2])
plt.title('filter')
im = plt.imshow(conv_filter, cmap='gray')
plt.colorbar(im, orientation='horizontal')



plt.gca().axes.set_axis_off()

#### Sensitivity of Image Features

In [ ]:
conv_layer_max = conv_layer.max()
def plot_conv_layer(tolerance):
    conv_layer_filtered = (conv_layer >= (conv_layer_max - tolerance)).astype(int)
    fig = plt.figure(figsize=(10, 10))
    im = plt.imshow(conv_layer_filtered, cmap='gray')
    plt.colorbar(im, orientation='horizontal')
    plt.gca().axes.set_axis_off()
    plt.show()
    
interactive(plot_conv_layer, 
            tolerance=ipw.FloatSlider(0.5, min=0, max=conv_layer_max - 0.1, step=0.01))

#### Interactive Convolution Filter
 - top
 - left
 - bottom
 - right
 - diag_left
 - diag_righ
 - top_half
 - bottom_half

In [ ]:
from utils.nn_visualization import conv_filter_widget
conv_filter_widget(image, conv_filter_shape=[8, 8], stride_col=1, stride_row=1, init_mode='top_half', padding='SAME')